In [8]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training, extract_face
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler, SequentialSampler
from torchvision import datasets, transforms
import numpy as np
import os
import pickle,pdb,math

data_dir = '../data/lfw/lfw'
pairs_path = '../data/lfw/pairs.txt'

batch_size = 2
epochs = 15
workers = 0 if os.name == 'nt' else 8

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('在该设备上运行: {}'.format(device))

with open('./embed_to_cls.pkl','rb') as file:
    embeddings_dict=pickle.load(file)

在该设备上运行: cpu


In [9]:
def read_pairs(pairs_filename):
    pairs = []
    with open(pairs_filename, 'r') as f:
        for line in f.readlines()[1:]:
            pair = line.strip().split()
            pairs.append(pair)
    return np.array(pairs, dtype=object)

def get_paths(lfw_dir, pairs):
    nrof_skipped_pairs = 0
    path_list = []
    issame_list = []
    for pair in pairs:  # pair=(str,str,str)
        # print(f"len={len(pair)}")
        path0=lfw_dir+'/'+pair[0]   # 当前图像地址填充
        path1=lfw_dir+'/'+pair[1]

        if pair[2]=="1":
            issame = True
        elif pair[2]=="0":
            issame = False

        if os.path.exists(path0) and os.path.exists(path1):    # 仅在两个路径都存在时添加配对
            path_list.append(path0)
            path_list.append(path1)
            issame_list.append(issame)
        else:
            nrof_skipped_pairs += 1
    if nrof_skipped_pairs>0:
        print('跳过 %d 个图像对' % nrof_skipped_pairs)

    return path_list, issame_list

In [10]:
# 以下是从David Sandberg的FaceNet实现中提取的LFW函数
def distance(embeddings1, embeddings2, distance_metric=0):
    if distance_metric==0:
        # Euclidian distance
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),1)
    elif distance_metric==1:
        # 基于余弦相似度的距离
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        similarity = dot / norm
        dist = np.arccos(similarity) / math.pi
    else:
        raise 'Undefined distance metric %d' % distance_metric

    return dist

pairs = read_pairs(pairs_path)
path_list, issame_list = get_paths(data_dir+'_cropped', pairs)

embeddings = np.array([embeddings_dict[path] for path in path_list if path in embeddings_dict.keys() ])

跳过 5 个图像对


In [18]:
embeddings1 = embeddings[0::2]
embeddings2 = embeddings[1::2]
# print(embeddings1[0])
print(embeddings1[0].shape)
print(len(embeddings2[0]))
pair_dist=distance(embeddings1[:4],embeddings2[:4],distance_metric=1)
print(f"dist={pair_dist},tag={issame_list[:4]}")


(512,)
512
dist=[0.12403172 0.18918154 0.16397257 0.18610983],tag=[False, False, False, False]
